In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Voting classifier

<img src=http://rasbt.github.io/mlxtend/user_guide/classifier/EnsembleVoteClassifier_files/majority_voting.png width=400/>

</br>

**Метод простого голосования**
* $c_1, c_2, ..., c_m$ - обученные алгоритмы  
* Классификация: относим $x$ к классу, за который проголосовало *большинство* из $c_1(x), c_2(x), ..., c_m(x)$
* Регрессия: ответом является среднее значение $c_1(x), c_2(x), ..., c_m(x)$



In [ ]:
from sklearn.datasets import make_blobs
X1, y1 = make_blobs(n_samples=1000, centers=[[-2, 1], [0, -2]], cluster_std=1)
X2, y2 = make_blobs(n_samples=500, centers=[[3, 0], [1, 3]], cluster_std=1)
X = np.concatenate([X1, X2])
y = np.concatenate([y1, y2])

In [ ]:
def plot_blobs(x, y):

    fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    reds = y == 0
    blues = y == 1
    
    ax.scatter(x[reds, 0], x[reds, 1], c="red", s=80, edgecolor="K")
    ax.scatter(x[blues, 0], x[blues, 1], c="blue", s=80, edgecolor="K")

    plt.show()

plot_blobs(X, y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
X_train.shape, X_test.shape

## Логистическая регрессия

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
confusion_matrix(y_test, y_pred)

In [ ]:
def plot_confusion(model, X, y, class_names=('red', 'blue')):
    ConfusionMatrixDisplay.from_estimator(model, X, y, cmap=plt.cm.Blues, display_labels=class_names, values_format='d')
plot_confusion(lr, X_train, y_train)
plot_confusion(lr, X_test, y_test)

In [ ]:
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
def plot_blob_classes(model, X, y):
    plt.figure(figsize=(15,8))

    eps = 0.1
    xx, yy = np.meshgrid(np.linspace(np.min(X[:,0]) - eps, np.max(X[:,0]) + eps, 500),
                         np.linspace(np.min(X[:,1]) - eps, np.max(X[:,1]) + eps, 500))

    colors = ("red", "blue")
    colored_y = np.zeros(y.size, dtype=str)

    for i, cl in enumerate([0,1]):
        colored_y[y == cl] = str(colors[i])

    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])

    Z = Z.reshape(xx.shape)

    cmap_light = ListedColormap(['#FFAAAA', 'grey'])
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)
    plt.scatter(X[:, 0], X[:, 1], c=colored_y, edgecolors='K')
    plt.show()

plot_blob_classes(lr, X, y)

In [ ]:
lr.score(X_test, y_test)

## Дерево решений

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=2)
dt.fit(X_train, y_train)
plot_confusion(dt, X_test, y_test)
plot_blob_classes(dt, X, y)
dt.score(X_test, y_test)

In [ ]:
dt3 = DecisionTreeClassifier(max_depth=3)
dt3.fit(X_train, y_train)
plot_confusion(dt3, X_test, y_test)
plot_blob_classes(dt3, X, y)
dt3.score(X_test, y_test)

## K-NN 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
plot_confusion(knn, X_test, y_test)
plot_blob_classes(knn, X, y)

knn.score(X_test, y_test)

## Наивный байесовский классификатор

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train, y_train)
plot_confusion(gnb, X_test, y_test)
plot_blob_classes(gnb, X, y)
gnb.score(X_test, y_test)

## Voting classifier

In [ ]:
classifiers = [('lr', lr), ('dt2', dt), ('dt3', dt3), ('knn', knn), ('gnb', gnb)]
classifiers

In [ ]:
from sklearn.ensemble import VotingClassifier

vc = VotingClassifier(estimators=classifiers, voting='hard')
vc.fit(X_train, y_train)
plot_confusion(vc, X_test, y_test)
plot_blob_classes(vc, X, y)
vc.score(X_test, y_test)

In [ ]:
params = {'lr__C' : [0.5, 1, 1.5], 
          'lr__class_weight': [None,'balanced'],
          'dt2__max_depth' : np.arange(1, 6),
          'dt3__max_depth' : np.arange(1, 6),
          'knn__n_neighbors' : np.arange(1, 20),
          'voting' : ['hard', 'soft']} 
np.prod([len(v) for k, v in params.items()])

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
optimal = RandomizedSearchCV(estimator=vc, param_distributions=params, n_iter=500, cv=5, scoring='accuracy', n_jobs=-1).fit(X_train, y_train)

In [ ]:
optimal.best_params_

In [ ]:
plot_confusion(optimal, X_test, y_test)
plot_blob_classes(optimal, X, y)
optimal.score(X_test, y_test)

# Случайный лес


**Бутстрэп**  
<img src=https://cdn.analyticsvidhya.com/wp-content/uploads/2020/02/Bagging.png width=700/>

</br>

<image src=https://всеответы.рф/wp-content/uploads/2020/03/Random-forest.jpg width=800 />

## Пятна

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(min_samples_split=10).fit(X_train, y_train)
plot_confusion(rf, X_test, y_test)
plot_blob_classes(rf, X, y)
rf.score(X_test, y_test)

## Титаник

In [ ]:
!wget -c https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv
!ls

In [ ]:
df = pd.read_csv('titanic.csv')
df

In [ ]:
yt = df.Survived.values
Xt = df.drop(['Name', 'Survived'], axis=1)
Xt.sample(5)

In [ ]:
Xt['Sex'].unique()

In [ ]:
categories = {'female' : 1, 'male' : 0}
Xt['Sex'] = Xt['Sex'].map(categories)
Xt.sample(5)

In [ ]:
from sklearn.model_selection import train_test_split
Xt_train, Xt_test, yt_train, yt_test = train_test_split(Xt, yt, test_size=0.25)

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier().fit(Xt_train, yt_train)

In [ ]:
plot_confusion(rf, Xt_test, yt_test, class_names=['died', 'survived'])

In [ ]:
rf.score(Xt_test, yt_test)

In [ ]:
sns.barplot(x=rf.feature_importances_, y=Xt_train.columns)

#Boosting
<image src="https://upload.wikimedia.org/wikipedia/commons/b/b5/Ensemble_Boosting.svg" alt="boosting diagram" width=800/>

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ab = AdaBoostClassifier().fit(X_train, y_train)
plot_blob_classes(ab, X, y)
ab.score(X_test, y_test)

In [ ]:
ab = AdaBoostClassifier().fit(Xt_train, yt_train)
sns.barplot(x=ab.feature_importances_, y=Xt_train.columns)
ab.score(Xt_test, yt_test)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier().fit(X_train, y_train)
plot_blob_classes(gb, X, y)
gb.score(X_test, y_test)

In [ ]:
gb = GradientBoostingClassifier().fit(Xt_train, yt_train)
sns.barplot(x=gb.feature_importances_, y=Xt_train.columns)
gb.score(Xt_test, yt_test)

# Перцептрон

In [ ]:
from sklearn.datasets import make_blobs
X1, y1 = make_blobs(n_samples=1000, centers=[[-2, 1], [0, -2]], cluster_std=1)
X2, y2 = make_blobs(n_samples=500, centers=[[3, 0], [1, 3]], cluster_std=1)
X = np.concatenate([X1, X2])
y = np.concatenate([y1, y2])
plot_blobs(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

## Keras

[Keras](https://keras.io/) — библиотека, предоставляющая высокоуровневые cтроительные блоки для конструирования моделей глубокого обучения.  

|![Keras](https://www.digikey.com/maker-media/6c3d4f5f-98e0-4104-ad8c-fb0b47000109 "Title")|
|:-------------------------------:|
| Source: https://www.digikey.com/maker-media/6c3d4f5f-98e0-4104-ad8c-fb0b47000109 |

### Определение модели

Модель можно определить двумя способами:   
1. с использованием класса Sequential (только для линейного стека слоев) 
2. с помощью функционального API (для ориентированного ациклического графа слоев, позволяющего конструировать произвольные архитектуры). Функциональный API позволяет манипулировать данными в тензорах, которые обрабатывает модель, и применять слои к этим тензорам, как если бы они были функциями.

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential()
model.add(Input(shape=(2,)))
model.add(Dense(4, activation='elu'))
model.add(Dense(4, activation='elu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
# input_tensor = Input(shape=(2,))
# d1 = Dense(4, activation='elu')(input_tensor)
# d2 = Dense(4, activation='elu')(d1)
# output_tensor = Dense(1, activation='sigmoid')(d2)

# model = Model(inputs=input_tensor, outputs=output_tensor)
# model.summary()

In [ ]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

### Обучение модели

Процесс обучения состоит в передаче массивов Numpy с входными данными (и соответствующими целевыми данными) в метод fit()

Сеть перебирает обучающие данные пакетами по 32 образца. Для каждого пакета сеть вычисляет градиенты весов с учетом потерь в пакете и изменяет значения весов в соответствующем направлении.

Можно разделить полный датасет из 1200 объектов на батчи размером 400 объектов. Таким образом, для завершения одной эпохи потребуется 3 итерации.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

results = model.fit(X_train, 
                    y_train,
                    batch_size=1200,
                    epochs=200000,
                    validation_split=0.2,
                    shuffle=True,
                    callbacks=[EarlyStopping(monitor='val_loss',
                                             patience=3,
                                             restore_best_weights=True)]
                    )

### Объект history

Вызов `model.fit()` возвращает объект History. Этот объект имеет поле *history* - словарь со значениями функции потерь и метрики 

In [ ]:
results

In [ ]:
results.history.keys()

In [ ]:
hist = pd.DataFrame(results.history)
hist

In [ ]:
hist[['loss']].plot()
hist[['binary_accuracy']].plot()

In [ ]:
loss, acc = model.evaluate(X_test, y_test, batch_size=300)
print('Test loss:', loss)
print('Test accuracy:', acc)

In [ ]:
plot_blob_classes(model, X, y)

# Задание

1. Применить один из бустовых классификаторов к своему датасету, кроссвалидацией оценить accuracy. Как она соотносится с accuracy из задания к семинару 3?
2. Визуализировать оценку важности признаков в обученном классификаторе.